In [1]:
import numpy as np


In [2]:
from pathlib import Path

WORKDIR = Path.cwd().absolute()
INPUTFILE = WORKDIR / "day-15.txt"

with INPUTFILE.open("r") as file:
    inputs = [line.strip() for line in file.readlines()]

_inputs = [
]

inputs[0:8]

['Sensor at x=3844106, y=3888618: closest beacon is at x=3225436, y=4052707',
 'Sensor at x=1380352, y=1857923: closest beacon is at x=10411, y=2000000',
 'Sensor at x=272, y=1998931: closest beacon is at x=10411, y=2000000',
 'Sensor at x=2119959, y=184595: closest beacon is at x=2039500, y=-250317',
 'Sensor at x=1675775, y=2817868: closest beacon is at x=2307516, y=3313037',
 'Sensor at x=2628344, y=2174105: closest beacon is at x=3166783, y=2549046',
 'Sensor at x=2919046, y=3736158: closest beacon is at x=3145593, y=4120490',
 'Sensor at x=16, y=2009884: closest beacon is at x=10411, y=2000000']

In [3]:
sensors = []
for report in inputs:
    split = report.split(" ")
    sx = int(split[2][2:-1])
    sy = int(split[3][2:-1])
    bx = int(split[8][2:-1])
    by = int(split[9][2:])
    sensors.append((sx, sy, bx, by))

print(sensors[0])
print(sensors[-1])


(3844106, 3888618, 3225436, 4052707)
(3448383, 3674287, 3225436, 4052707)


In [4]:
min_sx = min(map(lambda e: e[0], sensors))
max_sx = max(map(lambda e: e[0], sensors))
min_sy = min(map(lambda e: e[1], sensors))
max_sy = max(map(lambda e: e[1], sensors))
min_bx = min(map(lambda e: e[2], sensors))
max_bx = max(map(lambda e: e[2], sensors))
min_by = min(map(lambda e: e[3], sensors))
max_by = max(map(lambda e: e[3], sensors))

print(f"({min_sx},{min_sy}) - ({max_sx},{max_sy})")
print(f"({min_bx},{min_by}) - ({max_bx},{max_by})")


(16,130612) - (3995832,3988246)
(-175938,-250317) - (4552923,4120490)


In [5]:
#  |
#  |
#  |      .
#  |     / \
#  |    / C \
#  |   /     \
#  |  /       \
#  | . D  S  B .
#  |  \       /
#  |   \     /
#  |    \ A /
#  |     \ /
#  |      .
#  |
# -+------------
#  |

from collections import namedtuple

Point = namedtuple("Point", ["x", "y"])
Exclusion = namedtuple("Exclusion", ["A", "B", "C", "D"])

exclusions = []
for sensor in sensors:
    sx, sy, bx, by = sensor
    radius = abs(sx-bx)+abs(sy-by)
    exclusions.append(Exclusion(Point(sx,sy-radius),Point(sx+radius,sy),Point(sx,sy+radius),Point(sx-radius,sy)))

print(exclusions[0])
print(exclusions[-1])


Exclusion(A=Point(x=3844106, y=3105859), B=Point(x=4626865, y=3888618), C=Point(x=3844106, y=4671377), D=Point(x=3061347, y=3888618))
Exclusion(A=Point(x=3448383, y=3072920), B=Point(x=4049750, y=3674287), C=Point(x=3448383, y=4275654), D=Point(x=2847016, y=3674287))


In [6]:
#  *
#  |\
#  | \
#  |  \
#  |   \
#  |    \
#  |     \
#  |      .
#  |     / \
#  |    / C \
#  *   /     \
#  |\ /       \
#  | . D  S  B .
#  |/ \       /
#  *   \     /
#  |    \ A /
#  |     \ /
#  |      .
#  |     /
# -+------------
#  |   /
#  |  /
#  | /
#  |/
#  *

Intercepts = namedtuple("Intercepts", ["AB", "DC", "DA", "CB"])

intercepts = []
for exclusion in exclusions:
    bx, by = exclusion.B
    dx, dy = exclusion.D
    intercepts.append(Intercepts(by-bx, dy-dx, dy+dx, by+bx))

print(intercepts[0])
print(intercepts[-1])


Intercepts(AB=-738247, DC=827271, DA=6949965, CB=8515483)
Intercepts(AB=-375463, DC=827271, DA=6521303, CB=7724037)


In [7]:
y = 2_000_000
Segment = namedtuple("Segment", ["start", "end"])
segments = []
for exclusion, intercept in zip(exclusions, intercepts):
    if (y < exclusion.A.y) or (y > exclusion.C.y):
        print(f"No overlap with {exclusion}")
        continue
    if y < exclusion.B.y:
        start = intercept.DA - y    # start = y and intercept.DA
        end   = y - intercept.AB    # end   = y and intercept.AB
    else:   # y > exclusion.B.y
        start = y - intercept.DC    # start = y and intercept.DC
        end   = intercept.CB - y    # end   = y and intercept.CB
    segment = Segment(start, end)
    print(f"New segment {segment} with exclusion {exclusion}")
    segments.append(segment)

print(segments)


No overlap with Exclusion(A=Point(x=3844106, y=3105859), B=Point(x=4626865, y=3888618), C=Point(x=3844106, y=4671377), D=Point(x=3061347, y=3888618))
New segment Segment(start=10411, end=2750293) with exclusion Exclusion(A=Point(x=1380352, y=345905), B=Point(x=2892370, y=1857923), C=Point(x=1380352, y=3369941), D=Point(x=-131666, y=1857923))
New segment Segment(start=-9867, end=10411) with exclusion Exclusion(A=Point(x=272, y=1987723), B=Point(x=11480, y=1998931), C=Point(x=272, y=2010139), D=Point(x=-10936, y=1998931))
No overlap with Exclusion(A=Point(x=2119959, y=-330776), B=Point(x=2635330, y=184595), C=Point(x=2119959, y=699966), D=Point(x=1604588, y=184595))
New segment Segment(start=1366733, end=1984817) with exclusion Exclusion(A=Point(x=1675775, y=1690958), B=Point(x=2802685, y=2817868), C=Point(x=1675775, y=3944778), D=Point(x=548865, y=2817868))
New segment Segment(start=1889069, end=3367619) with exclusion Exclusion(A=Point(x=2628344, y=1260725), B=Point(x=3541724, y=217410

In [8]:
from functools import cmp_to_key

segments = sorted(segments, key=cmp_to_key(lambda a, b: a.start - b.start if a.start != b.start else a.end - b.end))
segments

[Segment(start=-10379, end=10411),
 Segment(start=-9867, end=10411),
 Segment(start=10411, end=32751),
 Segment(start=10411, end=2750293),
 Segment(start=563442, end=1100270),
 Segment(start=1366733, end=1984817),
 Segment(start=1889069, end=3367619),
 Segment(start=2393367, end=2541377),
 Segment(start=2535371, end=3715829),
 Segment(start=3082762, end=4908902),
 Segment(start=3445903, end=4253073)]

In [9]:
consolidated = []
total = 0
current = None
for subsequent in segments:
    if current is None:
        current = subsequent
        continue
    if subsequent.start <= current.end:
        current = Segment(current.start, max(subsequent.end, current.end))
    else:
        total += current.end - current.start + 1
        consolidated.append(current)
        current = subsequent

total += current.end - current.start + 1
consolidated.append(current)
current = subsequent

beacons_on_y = len(set(map(lambda r: r[2], filter(lambda r: r[3] == y, sensors))))

print(f"Removing {beacons_on_y} from total...")
total -= beacons_on_y

# 4919278 too small
# 4919282 too large
print(f"Part 1: total = {total}")
print(consolidated)


Removing 1 from total...
Part 1: total = 4919281
[Segment(start=-10379, end=4908902)]


In [10]:
def compare(a:Exclusion, b:Exclusion) -> int:
    if a.A.x < b.A.x:
        return -1
    elif a.A.x > b.A.x:
        return 1
    if a.A.y < b.A.y:
        return -1
    elif a.A.y > b.A.y:
        return 1
    if a.D.x < b.D.x:
        return -1
    elif a.D.x > b.D.x:
        return 1
    if a.D.y < b.D.y:
        return -1
    elif a.D.y > b.D.y:
        return 1
    return 0

ordered = sorted(exclusions, key=cmp_to_key(compare))
for exclusion in ordered:
    print(exclusion)


Exclusion(A=Point(x=16, y=1989605), B=Point(x=20295, y=2009884), C=Point(x=16, y=2030163), D=Point(x=-20263, y=2009884))
Exclusion(A=Point(x=272, y=1987723), B=Point(x=11480, y=1998931), C=Point(x=272, y=2010139), D=Point(x=-10936, y=1998931))
Exclusion(A=Point(x=21581, y=1988830), B=Point(x=1276231, y=3243480), C=Point(x=21581, y=4498130), D=Point(x=-1233069, y=3243480))
Exclusion(A=Point(x=135817, y=948865), B=Point(x=613993, y=1427041), C=Point(x=135817, y=1905217), D=Point(x=-342359, y=1427041))
Exclusion(A=Point(x=831856, y=-1085446), B=Point(x=2508786, y=591484), C=Point(x=831856, y=2268414), D=Point(x=-845074, y=591484))
Exclusion(A=Point(x=1082213, y=2087734), B=Point(x=2702561, y=3708082), C=Point(x=1082213, y=5328430), D=Point(x=-538135, y=3708082))
Exclusion(A=Point(x=1380352, y=345905), B=Point(x=2892370, y=1857923), C=Point(x=1380352, y=3369941), D=Point(x=-131666, y=1857923))
Exclusion(A=Point(x=1675775, y=1690958), B=Point(x=2802685, y=2817868), C=Point(x=1675775, y=3944

In [19]:
def intersection_y(y:int) -> list:

    Segment = namedtuple("Segment", ["start", "end"])
    segments = []
    for exclusion, intercept in zip(exclusions, intercepts):
        if (y < exclusion.A.y) or (y > exclusion.C.y):
            # print(f"No overlap with {exclusion}")
            continue
        if y < exclusion.B.y:
            start = intercept.DA - y    # start = y and intercept.DA
            end   = y - intercept.AB    # end   = y and intercept.AB
        else:   # y > exclusion.B.y
            start = y - intercept.DC    # start = y and intercept.DC
            end   = intercept.CB - y    # end   = y and intercept.CB
        segment = Segment(start, end)
        # print(f"New segment {segment} with exclusion {exclusion}")
        segments.append(segment)

    return segments

def intersection_x(x:int) -> list:

    Segment = namedtuple("Segment", ["start", "end"])
    segments = []
    for exclusion, intercept in zip(exclusions, intercepts):
        if (x < exclusion.D.x) or (x > exclusion.B.x):
            # print(f"No overlap with {exclusion}")
            continue
        if x < exclusion.A.x:
            start = intercept.DA - x
            end   = intercept.DC + x
        else:   # x > exclusion.A.x
            start = intercept.AB + x
            end   = intercept.CB - x
        segment = Segment(start, end)
        # print(f"New segment {segment} with exclusion {exclusion}")
        segments.append(segment)

    return segments

def no_beacon(segments:list) -> int:

    ordered = sorted(segments, key=cmp_to_key(lambda a, b: a.start - b.start if a.start != b.start else a.end - b.end))
    consolidated = []
    total = 0
    current = None
    for subsequent in ordered:

        if subsequent.end < 0 or subsequent.start > 4_000_000:
            continue

        if subsequent.start < 0:
            subsequent = Segment(0, subsequent.end)
        if subsequent.end > 4_000_000:
            subsequent = Segment(subsequent.start, 4_000_000)

        if current is None:
            current = subsequent
            continue
        if subsequent.start <= current.end:
            current = Segment(current.start, max(subsequent.end, current.end))
        else:
            total += current.end - current.start + 1
            consolidated.append(current)
            current = subsequent

    total += current.end - current.start + 1
    consolidated.append(current)
    current = subsequent

    # beacons_on_y = len(set(map(lambda r: r[2], filter(lambda r: r[3] == y, sensors))))

    # # print(f"Removing {beacons_on_y} from total...")
    # total -= beacons_on_y

    # # print(consolidated)
    return total, consolidated

# print(no_beacon(intersection_y(0)))
# print(no_beacon(intersection_y(4_000_000)))
# print(no_beacon(intersection_x(0)))
# print(no_beacon(intersection_x(4_000_000)))

for y in range(4_000_001):
    total, segments = no_beacon(intersection_y(y))
    if total < 4_000_001:
        print(f"{y=}")
        print(f"{total=}")
        print(segments)


y=3363767
total=4000000
[Segment(start=0, end=3157534), Segment(start=3157536, end=4000000)]


In [20]:
print(f"{4_000_000*3_157_535+3_363_767}")

12630143363767


Bad pipe message: %s [b'\x9b\x911\xfe\xe80@\x8dP\x8e\xaa\xfeT=:G\xe7\xe0 \xcb\xab(~L\xcdA\xc7\x04\x92tK\x8e\x8b\x04\xf1Ij\x90\x95~\xe7\x89\xb6+"+j\x1cIv0\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04', b'\x01\x02']
Bad pipe message: %s [b'\xaeLD\x8e\x03\xc3P\x89\x9a\x90p\xa2\x19\x08\xe6\xf0\xc7\r\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0', b"\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0"]
Bad pipe message: %s [b'3\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0']
Bad pipe message: %s [b'=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00